<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/04_Medical_Cost_Polynomial_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## Data Preprocessing

## importing the dataset
df = pd.read_csv('/content/drive/My Drive/life-long learning/MyProjects/Medical Cost/cleaned_insurance.csv')

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges,bmi_status,AgeInterval
0,19,female,27.900,0,True,southwest,16884.92400,overweight,"(15, 25]"
1,18,male,33.770,1,False,southeast,1725.55230,obesity,"(15, 25]"
2,28,male,33.000,3,False,southeast,4449.46200,obesity,"(25, 45]"
3,33,male,22.705,0,False,northwest,21984.47061,ideal,"(25, 45]"
4,32,male,28.880,0,False,northwest,3866.85520,overweight,"(25, 45]"


In [0]:
X = df.iloc[:, :6].values
y = df.iloc[:, 6].values

In [5]:
print(X)

[[19 'female' 27.9 0 True 'southwest']
 [18 'male' 33.77 1 False 'southeast']
 [28 'male' 33.0 3 False 'southeast']
 ...
 [18 'female' 36.85 0 False 'southeast']
 [21 'female' 25.8 0 False 'southwest']
 [61 'female' 29.07 0 True 'northwest']]


In [6]:
print(y)

[16884.924   1725.5523  4449.462  ...  1629.8335  2007.945  29141.3603]


In [0]:
# encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1,4,5])], remainder='passthrough')
X = ct.fit_transform(X)

In [8]:
print(X[:5])

[[0.0 1.0 0.0 0.0 1.0 19 27.9 0]
 [1.0 0.0 0.0 1.0 0.0 18 33.77 1]
 [1.0 0.0 0.0 1.0 0.0 28 33.0 3]
 [1.0 0.0 1.0 0.0 0.0 33 22.705 0]
 [1.0 0.0 1.0 0.0 0.0 32 28.88 0]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [29]:
# Training the Polynomial Regression model on the Training set
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_feature = PolynomialFeatures(degree=2)
X_train_poly = poly_feature.fit_transform(X_train)
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
# Predicting the Test set results
X_test_poly = poly_feature.fit_transform(X_test)
y_pred = poly_reg.predict(X_test_poly)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[:10])

[[41938.75 43254.42]
 [ 2916.75  1253.94]
 [11236.39 10594.5 ]
 [46054.98 43896.38]
 [52638.35 45863.21]
 [12733.88 10141.14]
 [ 5638.89  5253.52]
 [ 7053.8   5377.46]
 [12040.46 25517.11]
 [ 1316.3   1621.34]]


In [31]:
# Model Evaluation
from sklearn.metrics import r2_score
n = X_test.shape[0]
p = X_test.shape[1]
r_square = r2_score(y_test, y_pred)
adj_r_square = 1 - (1 - r_square) * ((n - 1) / (n - p - 1)) 
print("R-square : ",r_square)
print("Adjusted R-square : ",adj_r_square)

R-square :  0.7895482155243982
Adjusted R-square :  0.7830477743050747


In [32]:
# Applying K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = poly_reg, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 74.87 %
Standard Deviation: 5.14 %
